In [ ]:
print("start import")
import time
time1 = time.time()
from BTNHGV2Loader import BTNHGDatasetClass, paramsClass
import os
import importlib
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import HeteroData
from torch.utils.data import TensorDataset
from torch_geometric.loader import DataLoader, NeighborLoader
from torch_geometric.data import Dataset
from sklearn.model_selection import train_test_split
time2 = time.time()
print("import time: ", time2 - time1)
print(f"当前时间: {time.strftime('%m-%d %H:%M:%S', time.localtime())}")

start import
start import
import used time:  7.525163412094116
当前时间: 11-17 01:05:40
当前时间: 11-17 01:05:40
import time:  7.5387349128723145
当前时间: 11-17 01:05:40


In [2]:
d=BTNHGDatasetClass()

#输出当前时间
print(f"当前时间: {time.strftime('%m-%d %H:%M:%S', time.localtime())}")


start construct HeteroData
start read data
读取数据用时: 3.1228792667388916
当前时间: 11-17 01:05:45
construct ID map and 构建节点特征矩阵
构建ID map and 节点特征矩阵用时: 1.3321189880371094
当前时间: 11-17 01:05:46
构建边关系
构建边关系用时: 11.838828563690186
当前时间: 11-17 01:05:58
给 address 节点加标签
给 address 节点加标签用时:  0.4066126346588135
address node types: ['address', 'coin', 'tx']
address:torch.Size([546649, 13])
coin:torch.Size([2266449, 1])
tx:torch.Size([1000468, 5])
edge types: [('address', 'addr_to_coin', 'coin'), ('tx', 'tx_to_coin', 'coin'), ('coin', 'coin_to_tx', 'tx')]
address-coin:torch.Size([2, 2266449])
tx-coin:torch.Size([2, 2266449])
coin-tx:torch.Size([2, 1438396])
address y shape: torch.Size([546649])
address y elements: 546649
当前时间: 11-17 01:05:58
当前时间: 11-17 01:05:58


In [8]:
# %load_ext autoreload
# %autoreload 1
# %aimport BTNHGV2Loader
%load_ext autoreload
%autoreload 2
from BTNHGV2Loader import BTNHGDatasetClass


train_loader1, test_loader2 = d.getTrainLoaderAndTestLoader(batch_size=12174, isResetSeed=True)
train_loader2, test_loader2 = d.getTrainLoaderAndTestLoader(batch_size=12174, isResetSeed=True)
# 取出第一个 batch
x1, y1 = next(iter(train_loader1))
x2, y2 = next(iter(train_loader2))

#输出x1.shape
print("x1 shape:", x1.shape)
#输出y1.shape
print("y1 shape:", y1.shape)
# 遍历比较
for i in range(x1.size(0)):
    # 判断 x1[i] 是否存在于 x2 中
    match = torch.any(torch.all(x2 == x1[i], dim=1))
    if not match:
        print(f"索引 {i} 不存在于 train_loader2 中")
        print("内容:", x1[i])
        break
    

# #输出x1的每一个向量是否包含在x2中

# diff_index = (x1 != x2).nonzero(as_tuple=True)[0][0]
# print("第一个不同的索引:", diff_index)
# print("x1 不同的内容:", x1[diff_index])
# print("x2 不同的内容:", x2[diff_index])
# #输出y1和y2第一个不同的索引和内容
# diff_index = (y1 != y2).nonzero(as_tuple=True)[0][0]
# print("第一个不同的索引:", diff_index)
# print("y1 不同的内容:", y1[diff_index])
# print("y2 不同的内容:", y2[diff_index])

#输出当前时间
print(f"当前时间: {time.strftime('%m-%d %H:%M:%S', time.localtime())}")
#输出当前时间
print(f"当前时间: {time.strftime('%m-%d %H:%M:%S', time.localtime())}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
start split dataset to train and test
randSeed: 383329928
划分数据集信息
训练集大小: 12174 (80.00%)
测试集大小: 3044 (20.00%)
划分数据集用时: 0.04285001754760742
当前时间: 11-17 01:21:11
start split dataset to train and test
randSeed: 383329928
划分数据集信息
训练集大小: 12174 (80.00%)
测试集大小: 3044 (20.00%)
划分数据集用时: 0.033959150314331055
当前时间: 11-17 01:21:11
x1 shape: torch.Size([12174, 13])
y1 shape: torch.Size([12174])
索引 380 不存在于 train_loader2 中
内容: tensor([9.0000e+00, 8.3373e+07, 0.0000e+00, 0.0000e+00, 8.3373e+07, 9.0000e+00,
        2.7646e+05, 2.7762e+05, 1.1580e+03,        nan,        nan,        nan,
        1.1580e+03])
当前时间: 11-17 01:21:12
当前时间: 11-17 01:21:12
